# Used car price prediction

# Objective

There is a huge demand for used cars in the Indian Market today.Cars4U is a budding tech start-up that aims to find footholes in this market.We have to come up with a pricing model that can effectively predict the price of used cars and can help the business in devising profitable strategies using differential pricing.

In [2]:
# !pip install statsmodels

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno 
import plotly.express as px
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor


In [4]:
data = pd.read_csv('used_cars_data.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: 'used_cars_data.csv'

In [ ]:
data

# Data Understanding

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
print ("\nUnique values :  \n\n", data.nunique())

In [ ]:
msno.bar(data)

Observations

This preview shows that some columns potentially have a lot of missingness New_Price has only 1006 values. 86 % values are missing

Price, which is a Target variable 17 % missing values.

Seats has only 53 values missing and number of seats can be one of key factor in deciding price.

Power and Engine has 46 missing values.

Mileage only has two values missing.

Mileage,Power,Engine,New_Price we know are quantitative variables but are of object dtype here and needs to to converted to numeric.

In [ ]:
num = data.select_dtypes('number').columns.tolist()
cat = data.select_dtypes('object').columns.tolist()

In [ ]:
num

In [ ]:
cat

In [ ]:
cat_col = [
    "Fuel_Type",
    "Location",
    "Transmission",
    "Seats",
    "Year",
    "Owner_Type",
    
]
for column in cat_col:
    print(data[column].value_counts())
    print("-" * 40)

Observations

Maximum car being sold have fuel type as Diesel.

Mumbai has highest numbers of car availabe for purchase.

5204 cars with Manual transmission are available for purchase.

Most of the cars are 5 seaters and First owned.

Years of car ranges form 1996- 2015

# Data Preprocessing

Processing Engine,Power ,Mileage columns¶

In [ ]:
data[['Engine','Power','Mileage']].sample(10)

In [ ]:
typeoffuel=['CNG','LPG']
data.loc[data.Fuel_Type.isin(typeoffuel)].head(10)

Power has some values as "nullbhp" .Mileage also has some observations as 0. For fuel type and CNG and LPG mileage is measured in km/kg where as for other type it is measured in kmpl. Since those units are in km for both of them no need of conversion . Dropping units from mileages,Engine and Power.

 Mileage

In [ ]:
data[data.Mileage.isnull()==True]

In [ ]:

data["Mileage"] = data["Mileage"].str.rstrip(" kmpl")
data["Mileage"] = data["Mileage"].str.rstrip(" km/g")


Engine

In [ ]:
data["Engine"] = data["Engine"].str.rstrip(" CC")

Power

In [ ]:

data["Power"] = data["Power"].str.rstrip(" bhp")
data["Power"]= data["Power"].replace(regex="null", value = np.nan)

In [ ]:
num=['Engine','Power','Mileage']
data[num].sample(20)

 values in Power and Mileage as 0.0 so verifying data for Engine, Power, Mileage. Will check once again after converting datatype

In [ ]:
data.query("Power == '0.0'")['Power'].count()

In [ ]:
data.query("Mileage == '0.0'")['Mileage'].count()

In [ ]:
data.loc[data["Mileage"]=='0.0','Mileage']=np.nan

In [ ]:
data.loc[data["Engine"]=='0.0','Engine'].count()

In [ ]:
data[num].nunique()

In [ ]:
data[num].isnull().sum()

46 missing value in Engine ,175 missing value in power and 83 missing value in mileage

Processing Seats

In [ ]:
data.query("Seats == 0.0")['Seats'].count()

In [ ]:
data.loc[3999,'Seats'] =np.nan

In [ ]:
import re

new_price_num = []

regex_power = "^\d+(\.\d+)? Lakh$"

for observation in data["New_Price"]:
    if isinstance(observation, str):
        if re.match(regex_power, observation):
            new_price_num.append(float(observation.split(" ")[0]))
        else:
           
            print(
                "The data needs furthur processing.mismatch ",
                observation,
            )
    else:
        new_price_num.append(np.nan)

In [ ]:
data

In [ ]:

new_price_num = []

for observation in data["New_Price"]:
    if isinstance(observation, str):
        if re.match(regex_power, observation):
            new_price_num.append(float(observation.split(" ")[0]))
        else:
            new_price_num.append(float(observation.split(" ")[0]) * 100)
    else:
        new_price_num.append(np.nan)

data["new_price_num"] = new_price_num
data.head(5)  

# Feature Enginering

converting datatype

In [ ]:
#converting object data type to category data type
data["Fuel_Type"] = data["Fuel_Type"].astype("category")
data["Transmission"] = data["Transmission"].astype("category")
data["Owner_Type"] = data["Owner_Type"].astype("category")
#converting int data type to float  
data["Mileage"] = data["Mileage"].astype(float)
data["Power"] = data["Power"].astype(float)
data["Engine"]=data["Engine"].astype(float)

In [ ]:
data.describe().T

In [ ]:
data['Current_year']=2023
data['Ageofcar']=data['Current_year']-data['Year']
data.drop('Current_year',axis=1,inplace=True)
data.head()

Processing Name column

Brands do play an important role in Car selection and Prices. So extracting brand names from the Name.

In [ ]:
filtered_data = data.dropna(subset=['Name'])
data['Name'] = filtered_data['Name']

In [ ]:
 data['Brand'] = data['Name'].str.split(' ').str[0] 
data['Model'] = data['Name'].str.split(' ').str[1] + data['Name'].str.split(' ').str[2]

In [ ]:
data.Brand.unique()

In [ ]:
col=['ISUZU','Isuzu','Mini','Land']
data[data.Brand.isin(col)].sample(5)

Brand names like ISUZU and Isuzu are same and needs to be corrected. Land, Mini seems to be incorrect. So correcting brand names.

In [ ]:
data.info()

In [ ]:
data.loc[data.Brand == 'ISUZU','Brand']='Isuzu'
data.loc[data.Brand=='Mini','Brand']='Mini Cooper'
data.loc[data.Brand=='Land','Brand']='Land Rover'


In [ ]:
data.Brand.nunique()

In [ ]:
data.groupby(data.Brand).size().sort_values(ascending =False)

In [ ]:
data.Model.isnull().sum()

In [ ]:
data.dropna(subset=['Model'],axis=0,inplace=True)


In [ ]:
data.Model.nunique()

In [ ]:
data.groupby('Model')['Model'].size().nlargest(30)

There are 726 unique models and Swift Dzire is most popular Model.

# EDA

In [ ]:
data.info()

In [ ]:
data.describe()

Observations

Years is left skewed. Years ranges from 1996- 2019 . Age of cars 2 year old to 25 years old

Kilometer driven , median is ~53k Km and mean is ~58K. Max values seems to be 6500000. This is very high , and seems to be outlier. Need to analyze further.

Mileage is almost Normally distrubuited

Engine is right skewed and has outliers on higher and lower end

Power and Price are also right skewed.

Price 160 Lakh is too much for a used car. Seems to be an outlier.

In [ ]:
plt.style.use('ggplot')

numeric_columns = data.select_dtypes(include=np.number).columns.tolist()
plt.figure(figsize=(20,25))

for i, variable in enumerate(numeric_columns):
                     plt.subplot(10,3,i+1)
                       
                     sns.distplot(data[variable],kde=False,color='blue')
                     plt.tight_layout()
                     plt.title(variable)


Observations

Year is left skewed and has outilers on lower side., This column can be dropped

Kilometer_driven is right skewed.

Mileage is almost Normally distrubuted. Has few outliers on upper and lower side. need to check further.

Engine ,power and price are right skewed and has outliers on upper side.

Age of car is right skewed.

In [ ]:
cat_columns=['Location','Fuel_Type','Transmission', 'Owner_Type', 'Brand'] 

plt.figure(figsize=(15,21))

for i, variable in enumerate(cat_columns):
                     plt.subplot(4,2,i+1)
                     order = data[variable].value_counts(ascending=False).index    
                     ax=sns.countplot(x=data[variable], data=data , order=order ,palette='viridis')
                     for p in ax.patches:
                           percentage = '{:.1f}%'.format(100 * p.get_height()/len(data[variable]))
                           x = p.get_x() + p.get_width() / 2 - 0.05
                           y = p.get_y() + p.get_height()
                           plt.annotate(percentage, (x, y),ha='center')
                     plt.xticks(rotation=90)
                     plt.tight_layout()
                     plt.title(variable)

Observations

Car Profile

71 % cars available for sell have manual Transmission.

82 % cars are First owned cars.

39% of car available for sale are from Maruti & Hyundai brands.

53% of car being sold/avialable for purchase have fuel type as Diesel .

Mumbai has highest numbers of car availabe for purchase whereas Ahmedabad has least

Most of the cars are 5 seaters.

Car being sold/available for purchase are in 2 - 23 years old

71% car are lower price range car.

In [ ]:
numeric_columns= numeric_columns = data.select_dtypes(include=np.number).columns.tolist()
plt.figure(figsize=(13,17))

for i, variable in enumerate(numeric_columns):
                     plt.subplot(5,2,i+1)
                     sns.scatterplot(x=data[variable],y=data['Price']).set(title='Price vs '+ variable)
                     plt.tight_layout()

# Handling missing values

In [ ]:
data.isnull().sum()

In [ ]:
num_missing = data.isnull().sum(axis=1)
num_missing.value_counts()

In [ ]:

for n in num_missing.value_counts().sort_index().index:
    if n > 0:
        print("*" *30,f'\nFor the rows with exactly {n} missing values, NAs are found in:')
        n_miss_per_col = data[num_missing == n].isnull().sum()
        print(n_miss_per_col[n_miss_per_col > 0])
        print('\n\n')

In [ ]:
data[num_missing==7]

In [ ]:
col=['Engine','Power','Mileage']
data[col].isnull().sum()

In [ ]:
data.groupby(['Name','Year'])['Engine'].median().head(30)

In [ ]:
data['Engine']=data.groupby(['Name','Year'])['Engine'].apply(lambda x:x.fillna(x.median()))
data['Power']=data.groupby(['Name','Year'])['Power'].apply(lambda x:x.fillna(x.median()))
data['Mileage']=data.groupby(['Name','Year'])['Mileage'].apply(lambda x:x.fillna(x.median()))

In [ ]:
col=['Engine','Power','Mileage']
data[col].isnull().sum()

In [ ]:
data.groupby(['Brand','Model'])['Engine'].median().head(10)

In [ ]:
data.groupby(['Brand','Model'])['Engine'].median().head(10)

As we can see most of the model have same engine size and instead of just applying median , grouping with model and year that should give me more granularity, and near to accurate Engine values.

In [ ]:
data['Engine']=data.groupby(['Brand','Model'])['Engine'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data['Power']=data.groupby(['Brand','Model'])['Power'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data['Mileage']=data.groupby(['Brand','Model'])['Mileage'].apply(lambda x:x.fillna(x.median()))

In [ ]:
col=['Engine','Power','Mileage']
data[col].isnull().sum()

In [ ]:
data.groupby(['Model', 'Year'])['Engine'].agg(['median', 'mean', 'max']).sort_values(by='Model', ascending=True).head(10)

In [ ]:
data.groupby(['Brand','Engine'])['Power'].agg({'mean','median','max'}).head(10)

In [ ]:
data['Seats'].isnull().sum()

In [ ]:
data['Seats']=data.groupby(['Name'])['Seats'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data['Seats'].isnull().sum()

In [ ]:
data['Seats']=data.groupby(['Model'])['Seats'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data[data['Seats'].isnull()==True].head(10)

In [ ]:
#most of cars are 5 seater so fillrest of 23 by 5
data['Seats']=data['Seats'].fillna(5)

In [ ]:
data["Location"] = data["Location"].astype("category")
data['Brand'] =data['Brand'].astype("category")

In [ ]:
data.isnull().sum()

# Processing New Price

In [ ]:
data['new_price_num']=data.groupby(['Name','Year'])['new_price_num'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data['new_price_num']=data.groupby(['Name'])['new_price_num'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data['new_price_num']=data.groupby(['Brand','Model'])['new_price_num'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data['new_price_num']=data.groupby(['Brand'])['new_price_num'].apply(lambda x:x.fillna(x.median()))

In [ ]:
data.drop(['New_Price'],axis=1,inplace=True)

In [ ]:
data.new_price_num.isnull().sum()

In [ ]:
data.groupby(['Brand'])['new_price_num'].median().sort_values(ascending=False)

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
cols1 = ["Power","Mileage","Engine"]

for ii in cols1:
    data[ii] = data[ii].fillna(data[ii].median())

In [ ]:
data.dropna(inplace=True,axis=0)

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

Price

In [ ]:
data.groupby(['Brand'])['Price'].agg({'median','mean','max'})

In [ ]:
#using business knowledge to create class 
Low=['Maruti', 
     'Hyundai',
     'Ambassdor',
     'Hindustan',
     'Force',
     'Chevrolet',
     'Fiat',
     'Tata',
     'Smart',
     'Renault',
     'Datsun',
     'Mahindra',
     'Skoda',
     'Ford',
     'Toyota',
     'Isuzu',
     'Mitsubishi','Honda']
High=['Audi',
      'Mini Cooper',
      'Bentley',
      'Mercedes-Benz',
      'Lamborghini',
      'Volkswagen',
      'Porsche',
      'Land Rover',
      'Nissan',
      'Volvo',
      'Jeep',
      'Jaguar',
      'BMW']# more than 30lakh

In [ ]:
def classrange(x):
    if x in Low:
        return "Low"
    elif x in High:
        return "High"
    else: 
        return x

In [ ]:
data['Brand_Class'] = data['Brand'].apply(lambda x: classrange(x))

In [ ]:
data['Brand_Class'].unique()

In [ ]:
data['Engine']=data['Engine'].astype(int)
data['Brand_Class']=data["Brand_Class"].astype('category')

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data.corr(),annot=True ,cmap="YlGnBu" )
plt.show()

Observations

Engine has strong positive correlation to Power [0.86].

Price has positive correlation to Engine[0.66] as well Power [0.77].

Mileage is negative correlated to Engine,Power,Price.,Ageofcar

Price has negative correlation to age of car.

Kilometer driven doesnt impact Price

# Variables that are correlated with Price variable

In [ ]:
plt.figure(figsize=(10,7))

plt.title("Price VS Engine based on Transmission")
sns.scatterplot(y='Engine', x='Price', hue='Transmission', data=data)

In [ ]:
plt.figure(figsize=(10,7))
plt.title("Price vs Power based on Transmission")
sns.scatterplot(y='Power', x='Price', hue='Transmission', data=data)

In [ ]:
sns.scatterplot(y='Mileage', x='Price', hue='Transmission', data=data)

# Outlier Treatment

In [ ]:
num = data.select_dtypes(include=np.number).columns.tolist()
num.remove(
    "Price"
) 

In [ ]:
plt.figure(figsize=(17, 15))

for i, variable in enumerate(num):
    plt.subplot(2, 3, (i % 6) + 1)
    plt.boxplot(data[variable], whis=1.5)
    plt.tight_layout()
    plt.title(variable)

plt.show()

In [ ]:
def treat_outliers(df, col):
    """
    treats outliers in a variable
    col: str, name of the numerical variable
    df: dataframe
    col: name of the column
    """

    Q1 = df[col].quantile(0.25)  # 1st quantile
    Q3 = df[col].quantile(0.75)  # 3rd quantile
    IQR = Q3 - Q1
    Lower_whisker = Q1 - 1.5 * IQR
    Upper_whisker = Q3 + 1.5 * IQR

    df[col] = np.clip(df[col], Lower_whisker, Upper_whisker)

    return df
def treat_outliers_all(df, col_list):

    """
    treat outlier in all numerical variables
    col_list: list of numerical variables
    df: data frame
    """

    for c in col_list:
        df = treat_outliers(df, c)
    return df

In [ ]:
num.remove("Seats")

In [ ]:
data = treat_outliers_all(
    data, num
) 

In [ ]:
plt.figure(figsize=(17, 12))

for i, variable in enumerate(num):
    plt.subplot(2, 3, (i % 6) + 1)
    plt.boxplot(data[variable], whis=1.5)
    plt.tight_layout()
    plt.title(variable)

plt.show()

In [ ]:
data.shape

In [ ]:
cols_to_log = data.select_dtypes(include=np.number).columns.tolist()
for colname in cols_to_log:
    sns.distplot(data[colname], kde=True)
    plt.show()

In [ ]:
def Perform_log_transform(df,col_log):
    """#Perform Log Transformation of dataframe , and list of columns """
    for colname in col_log:
        df[colname + '_log'] = np.log(df[colname])
#     df.drop(col_log, axis=1, inplace=True)
    df.info()

In [ ]:
Perform_log_transform(data,['Kilometers_Driven','Price'])

In [ ]:
data.drop(['Name','Model','Year','Brand'],axis=1,inplace=True)

In [ ]:
#data.drop(['Name','Model','Year','Brand','new_price_num'],axis=1,inplace=True)

In [ ]:
data.drop(['Kilometers_Driven','Price'],axis=1,inplace=True)

In [ ]:
data.info()

In [ ]:
X = data.drop(["Price_log"], axis=1)
y = data["Price_log"]

In [ ]:
def encode_cat_vars(x):
    x = pd.get_dummies(
        x,
        columns=x.select_dtypes(include=["object", "category"]).columns.tolist(),
        drop_first=True,
    )
    return x

In [ ]:
X = encode_cat_vars(X)
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train.reset_index()
print("X_train:",X_train.shape)
print("X_test:",X_test.shape)
print("y_train:",y_train.shape)
print("y_test:",y_test.shape)

# Base Model

In [ ]:
import statsmodels.api as sm

In [ ]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)


def build_ols_model(train):
    # Create the model
    olsmodel = sm.OLS(y_train, X_train)
    return olsmodel.fit()


In [ ]:
olsmodel1 = build_ols_model(X_train)
print(olsmodel1.summary())

In [ ]:
X.columns

# OLS model after dropping insignificant feature

In [ ]:
X1 = X.drop(["Mileage","Seats","Location_Chennai","Location_Kochi","Fuel_Type_LPG","Owner_Type_Fourth & Above","Fuel_Type_Petrol","Power"], axis=1)

In [ ]:
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=42)

In [ ]:
X1_train = sm.add_constant(X1_train)
X1_test = sm.add_constant(X1_test)


def build_ols_model(train):
   
    olsmodel = sm.OLS(y_train, X1_train)
    return olsmodel.fit()

In [ ]:
olsmodel2 = build_ols_model(X1_train)
print(olsmodel2.summary())

# Test Assumptions of OLS

No Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor


def checking_vif(train):
    vif = pd.DataFrame()
    vif["feature"] = train.columns

    vif["VIF"] = [
        variance_inflation_factor(train.values, i) for i in range(len(train.columns))
    ]
    return vif

In [ ]:
print(checking_vif(X1_train))


Mean of residuals should be 0

In [ ]:
residuals = olsmodel2.resid
np.mean(residuals)

No Heteroscedasticity

In [ ]:
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(residuals, X1_train)
lzip(name, test)

Normality of error terms

In [ ]:
fitted = olsmodel2.fittedvalues
sns.residplot(x=fitted, y=residuals, color="purple", lowess=True)
plt.xlabel("Fitted Values")
plt.ylabel("Residuals")
plt.ylabel("Residuals")
plt.title("Residual PLOT")
plt.show()



 Normality of error terms

In [ ]:
sns.distplot(residuals)

In [ ]:
# Plot q-q plot of residuals
import pylab
import scipy.stats as stats

stats.probplot(residuals, dist="norm", plot=pylab)
plt.show()

# Lineara Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.scatter(y_test, y_pred)

In [ ]:
r2 = metrics.r2_score(y_test,y_pred)

In [ ]:
r2

In [ ]:
import math
rmse = math.sqrt(metrics.mean_squared_error(y_test,y_pred))

In [ ]:
rmse

# Decesion Tree Regressor

In [ ]:
DR = DecisionTreeRegressor()

In [ ]:
DR.fit(X_train, y_train)

In [ ]:
y_pred = DR.predict(X_test)

In [ ]:
plt.scatter(y_test, y_pred)

In [ ]:
r2_DR = metrics.r2_score(y_test, y_pred)
r2_DR

In [ ]:
rmse_DR = math.sqrt(metrics.mean_squared_error(y_test, y_pred))
rmse_DR

# Randomforest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RF = RandomForestRegressor()

In [ ]:
RF.fit(X_train, y_train)

In [ ]:
y_pred = RF.predict(X_test)

In [ ]:
r2_RF = metrics.r2_score(y_test, y_pred)
r2_RF

In [ ]:
plt.scatter(y_test, y_pred)

In [ ]:
rmse_RF = math.sqrt(metrics.mean_squared_error(y_test, y_pred))
rmse_RF

# Lasso Rigression

In [ ]:
from sklearn.linear_model import Lasso


In [ ]:
lasso=Lasso()
lasso.fit(X1_train,y_train)

In [ ]:
y_pred=lasso.predict(X1_test)

In [ ]:
y_pred = lasso.predict(X1_test)

In [ ]:
r2_lasso = metrics.r2_score(y_test, y_pred)
r2_lasso

In [ ]:
rmse_lasso = math.sqrt(metrics.mean_squared_error(y_test, y_pred))
rmse_lasso

# Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge=Ridge()
ridge.fit(X1_train,y_train)

In [ ]:
y_pred = ridge.predict(X1_test)

In [ ]:
r2_ridge = metrics.r2_score(y_test, y_pred)
r2_ridge

In [ ]:
rmse_ridge = math.sqrt(metrics.mean_squared_error(y_test, y_pred))
rmse_ridge